In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

2023-05-23 17:05:09.725767: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the dataset
data_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/'
dataset = np.load(data_path + 'PCA_dataset1.npy', allow_pickle=True)[()]

x_train = dataset['train_stars_pca']
x_val = dataset['validation_stars_pca']
x_test = dataset['test_stars_pca']
y_train = dataset['train_C']
y_val = dataset['validation_C']
y_test = dataset['test_C']
SED_test = dataset['test_SEDs']


In [6]:
# Hyperparameters
PCA_components = 24
model_learning_rate = 0.1
N_epochs = 50
N_committee = 48

In [4]:
def SEDlisttoC(SED_list):
    sed_array = np.array(SED_list)
    return sed_array*0.5 + 1.5

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

In [ ]:
example_SED_original = np.random.randint(low = 0, high = 13, size = 50)
example_C = SEDlisttoC(example_SED_original)
example_C_noisy = example_C + np.random.rand(example_C.shape[0])*0.8-0.40
example_SED_obtained = CtoSEDarray(example_C_noisy)
for i in range(example_SED_original.shape[0]):
    print(example_SED_original[i], example_C_noisy[i], example_SED_obtained[i])


In [7]:
# Define the model architecture
def create_model():
    initializer = tf.keras.initializers.GlorotNormal(seed = 1)
    model = tf.keras.Sequential([
        layers.Dense(26, input_shape=[PCA_components], activation='sigmoid', kernel_initializer= initializer),
        layers.Dense(26, activation='sigmoid', kernel_initializer= initializer),
        layers.Dense(1, activation = 'linear', kernel_initializer= initializer)
    ])
    model.compile(
        loss = tf.keras.losses.MeanSquaredError(),
        optimizer = tf.keras.optimizers.Adam(learning_rate = model_learning_rate)
    )
    return model

# Train a committee of 48 neural networks and make predictions
committee = []
committee_predictions = []
for i in range(N_committee):
    model = create_model()
    model.fit(x_train, y_train, epochs= N_epochs, verbose=0, validation_data=(x_val,y_val))
    committee.append(model)
    committee_predictions.append(model.predict(x_test, verbose = 1).reshape(-1)) # Predict the scalar parameter C using the committee    

committee_predictions = np.array(committee_predictions)
y_pred = np.mean(committee_predictions, axis=0)
pred_variance = np.var(committee_predictions, axis=0)
SED_pred = CtoSEDarray(y_pred,pred_variance)
#add metrics


1/1 [==============================] - 0s 51ms/step


In [8]:
 tf.math.confusion_matrix(SED_test, SED_pred) 

<tf.Tensor: shape=(12, 12), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]], dtype=int32)>

In [9]:
# Evaluate the performance of the committee

mse = np.mean((y_test - y_pred)**2)
print('MSE:', mse)
print(pred_variance)


for i in range(len(y_pred)):
    print(SED_test[i], y_pred[i], SED_pred[i])


MSE: 2.5850226317367286
[5.7506591e-12 5.5801288e-12 4.0264090e-12 7.5791222e-12 7.4038553e-12
 7.9343936e-12 7.0391102e-12 3.7706136e-12 4.9974838e-12 9.7865422e-12]
0 4.6000323 6
11 4.6000323 6
4 4.6000323 6
7 4.6000323 6
6 4.6000323 6
3 4.6000323 6
3 4.6000323 6
4 4.6000323 6
9 4.6000323 6
8 4.6000323 6
